CONJUNTO VERÓNICA


In [ ]:
#LIBRERIAS----
libraries <- c("tidyverse",
               "sf", 
               "tmap", 
               "utils", 
               "readr")


installed_packages <- rownames(installed.packages())
libraries_to_install <- setdiff(libraries, installed_packages)

if (length(libraries_to_install) > 0) {
  install.packages(libraries_to_install)
}

lapply(libraries, library, character.only = TRUE)

In [ ]:
##SITIO DE INTERÉS
sitio <- st_as_sf(data.frame(lon = -99.172436, lat = 19.438515), 
                  coords = c("lon", "lat"), 
                  crs = 4326)|>
  st_transform(32614)


#st_write(sitio, "sitio.geojson")

buffer<- sitio|>
  st_buffer(500)|>
  mutate(sitio = "zona influencia", 
         distancia = 500)
  
##COLONIAS Y ZONA DE INTERÉS ----
colonias <- st_read("https://gitlab.com/claudiodanielpc/colonias_mx/-/raw/main/iecm_colonias_cdmx.geojson")|>
  st_transform(32614)|>
  select(UT, CVEUT, DEMARCACIO)

colonia_sitio<-sitio|>
  st_intersection(colonias)

colonias_interes<-buffer|>
  st_intersection(colonias)|>
  st_drop_geometry()|>
  as.data.frame()

zona_interes <- colonias %>%
  filter(CVEUT %in% colonias_interes$CVEUT) %>%
  mutate(area = as.numeric(st_area(geometry)) / 10000)

  #st_write(zona_interes, "zona_interes.geojson")

In [ ]:
##DATOS DE VIVIENDA EN LAS COLONIAS
##PRINCIPALES RESULTADOS POR MANZANA CENSO DE POBLACIÓN Y VIVIENDA 2020
url <- "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/ageb_manzana/RESAGEBURB_09_2020_xlsx.zip"
temp_zip <- tempfile(fileext = ".zip")
temp_dir <- tempdir()
options(timeout = 800)
download.file(url, temp_zip, mode = "wb")
unzip(temp_zip, exdir = temp_dir)

RESAGEBURB_09XLSX20_BASE <- list.files(temp_dir, pattern ="RESAGEBURB_09XLSX20.xlsx", full.names = TRUE, recursive = TRUE)
RESAGEBURB_09XLSX20<- readxl::read_excel(RESAGEBURB_09XLSX20_BASE)|>
  mutate(CVEGEO = paste0(ENTIDAD, MUN, LOC, AGEB, MZA),
         CVE_MUNI = paste0(ENTIDAD, MUN))|>
  mutate_all(~ gsub("//*", "", .)) |> 
  mutate(across(9:230, ~ as.numeric(., na.rm = TRUE)))

##MANZANAS DEL MARCO GEOESTADISTICO NACIONAL 2020----
url <- "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/09_ciudaddemexico.zip"
temp_zip <- tempfile(fileext = ".zip")     
temp_dir <- tempdir()                       
options(timeout = 1000)                      
download.file(url, temp_zip, mode = "wb")
related_files <- paste0("conjunto_de_datos/09m", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])

manzanas <- st_read(shp_file_path, quiet = TRUE)|>
  st_transform(crs = 32614)|>
  st_centroid()|>
  st_intersection(zona_interes)

manzanas_censo_2020<- manzanas|>
  left_join(RESAGEBURB_09XLSX20, by = "CVEGEO")

#st_write(manzanas_censo_2020, "C:/Users/brenp/Desktop/PREDIO MH/manzanas_censo_2020.geojson")

##VIVIENDA TOTAL---
vivienda_total <- manzanas_censo_2020 |>
  st_drop_geometry()|>
  as.data.frame()|>
  group_by(UT) |>
  summarise(POB_TOTAL = sum(POBTOT, na.rm = TRUE),
            VIVIENDA_TOTAL = sum(VIVTOT, na.rm = TRUE),
            TOT_VIVPARHAB = sum(TVIVPARHAB, na.rm = TRUE),
            TOT_VIPAR = sum(TVIVPAR, na.rm = TRUE),
            TOT_VIV_DES = sum(VIVPAR_DES, na.rm = TRUE),
            PRC_VIV_DES = round(TOT_VIV_DES * 100 / TOT_VIPAR, 1))

# Calcular totales generales
totales <- manzanas_censo_2020 |>
  mutate(zona = "total")|>
  group_by(zona)|>
  summarise(POB_TOTAL = sum(POBTOT, na.rm = TRUE),
            VIVIENDA_TOTAL = sum(VIVTOT, na.rm = TRUE),
            TOT_VIVPARHAB = sum(TVIVPARHAB, na.rm = TRUE),
            TOT_VIPAR = sum(TVIVPAR, na.rm = TRUE),
            TOT_VIV_DES = sum(VIVPAR_DES, na.rm = TRUE),
            PRC_VIV_DES = round(TOT_VIV_DES * 100 / TOT_VIPAR, 1))

# Unir con los datos originales
tabla_completa <- bind_rows(vivienda_total, totales)|>
  st_drop_geometry()|>
  as.data.frame()|>
  left_join(zona_interes, by = "UT")|>
  mutate(dens_pob = POB_TOTAL/area)

print(tabla_completa)

In [ ]:
##DATOS DE AIRBNB----
relacion_viv_airbnb <- read.csv("C:/Users/brenp/Documents/GitHub/airbnb_stuff/clasificacion_colonias_tensionadas.csv")|>
  #select(cveut, ut, relvivairbnb)|>
  filter(cveut %in%zona_interes$CVEUT)

#LA BASE SE FILTRA POR LOS DATOS DE UNIDADES COMPLETAS EN RENTA
url <- "https://data.insideairbnb.com/mexico/df/mexico-city/2024-12-27/data/listings.csv.gz"

airbnb_base<-read_csv(url, col_types = cols(.default = col_character()))|>
  mutate(price = as.numeric(gsub("[$,]", "", price)),
         latitude= as.numeric(latitude), 
         longitude= as.numeric(longitude))|>
  rename(ID_AIRBNB=id)|>
  filter(room_type == "Entire home/apt")|>
  st_as_sf(coords = c("longitude", "latitude"), crs = 4326)|>
  st_transform(crs = 32614)|>
  st_intersection(zona_interes)

#st_write(airbnb_base, "airbnb_base_zona.geojson")


In [ ]:
##AREA DE INFLUENCIA DE LÍNEAS/ESTACIONES DE TRANSPORTE MASIVO ----
##METRO
url<- "https://datos.cdmx.gob.mx/dataset/1b014317-ddb1-46c7-ac79-7330c652abe3/resource/288b10dd-4f21-4338-b1ed-239487820512/download/stcmetro_shp.zip"
temp_zip <- tempfile(fileext = ".zip")     
temp_dir <- tempdir()

download.file(url, temp_zip, mode = "wb")

related_files <- paste0("stcmetro_shp/STC_Metro_lineas_utm14n", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])

metro_linea <- st_read(shp_file_path, quiet = TRUE)|>
  st_intersection(zona_interes)|>
  mutate(SISTEMA = "Metro")|>
  select(SISTEMA, RUTA)

##METROBUS
url<-"https://datos.cdmx.gob.mx/dataset/63a08f93-e959-430f-8dc4-bf07182401e6/resource/66cd5773-d83f-4a80-8c15-8ab34851e046/download/mb_shp.zip"
temp_zip <- tempfile(fileext = ".zip")     
temp_dir <- tempdir()

download.file(url, temp_zip, mode = "wb")

related_files <- paste0("mb_shp/Metrobus_lineas", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])

metrobus_lineas <- st_read(shp_file_path, quiet = TRUE)|>
  st_transform(crs = 32614)|>
  st_intersection(zona_interes)|>
  mutate(SISTEMA = "Metrobus")|>
  select(SISTEMA, RUTA)

related_files <- paste0("mb_shp/Metrobus_estaciones", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])

metrobus_punto <- st_read(shp_file_path, quiet = TRUE)|>
  st_transform(crs = 32614)|>
  st_intersection(zona_interes)|>
  mutate(SISTEMA = "Metrobus")|>
  select(SISTEMA, LINEA, NOMBRE)

##sistema de transporte electrico
url <- "https://datos.cdmx.gob.mx/dataset/6a3bc652-a775-41a0-aeb6-05d31ee40b59/resource/c7a1b906-92ff-4525-9b0c-f166ea731f95/download/ste_shp.zip"
temp_zip <- tempfile(fileext = ".zip")     
temp_dir <- tempdir()
download.file(url, temp_zip, mode = "wb")

##ESTACIONES DE TRANSPORTE ELECTRICO (TROLEBUS)
related_files <- paste0("ste_shp/ste_trolebus_shp/STE_Trolebus_Paradas", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])

trole<- st_read(shp_file_path, quiet = TRUE)|>
  st_transform(crs = 32614)|>
  mutate(SISTEMA = "TROLEBUS")|>
  st_intersection(zona_interes)|>
  select(SISTEMA, LINEA, NOMBRE)

##TROLEBUS LINEAS
related_files <- paste0("ste_shp/ste_trolebus_shp/STE_Trolebus_Lineas", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])

trole_lineas<- st_read(shp_file_path, quiet = TRUE)|>
  st_transform(crs = 32614)|>
  mutate(SISTEMA = "TROLEBUS")|>
  st_intersection(zona_interes)|>
  select(SISTEMA, RUTA)

##TRANSPORTE CONCESIONADO 
url <- "https://datos.cdmx.gob.mx/dataset/rutas-y-corredores-del-transporte-publico-concesionado/resource/6e0ec632-49bd-4f7b-a8f8-2afb44464ae1"
temp_zip <- tempfile(fileext = ".zip")     
temp_dir <- tempdir()
download.file(url, temp_zip, mode = "wb")

related_files <- paste0("concesionado_shp/concesionado_rutas", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])
camion_lineas<- st_read(shp_file_path, quiet = TRUE)|>
  st_transform(crs = 32614)|>
  mutate(SISTEMA = "Transporte concesionado")|>
  st_intersection(zona_interes)|>
  select(SISTEMA, RUTA)

##PARADAS DE TRANSPORTE CONCESIONADO
related_files <- paste0("concesionado_shp/concesionado_paradas", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])

camion_puntos<- st_read(shp_file_path, quiet = TRUE)|>
  st_transform(crs = 32614)|>
  mutate(SISTEMA = "Transporte concesionado")|>
  st_intersection(zona_interes)|>
  select(SISTEMA, RUTA)

##UNIR DATOS DE TRANSPORTE PÚBLICO
#LINEAS DE TRANSPORTE PÚBLICO
stpm_lineas <-metro_linea|>
  bind_rows(trole_lineas, metrobus_lineas, camion_lineas)

##ESTACIONES DE TRANSPORTE PÚBLICO
stpm_puntos <- metrobus_punto|>
  bind_rows(trole, camion_puntos)


#st_write(stpm_lineas, "stpm_lineas.geojson")
#st_write(stpm_puntos, "stpm_puntos.geojson")

In [ ]:
#ESPACIO PÚBLICO----
ep <- st_read("https://datos.cdmx.gob.mx/dataset/33f7efb2-540a-41e5-a7b2-f5c83e030b54/resource/e443a870-08d2-42ad-843f-ac4e0eb5541e/download/inventario-de-reas-verdes-en-la-ciudad-de-mxico..json") |>
  st_transform(crs = 32614) |>
  filter(categoria_ %in% c("Parques, arboledas y alamedas", "Plazas y jardines", "Forestación urbana", "Áreas verdes complementarias o ligadas a la red vial"))|>
  st_intersection(zona_interes)

ep_tipo <-ep|>
  group_by(categoria_)|>
  summarise(cuenta = n())

#st_write(ep,"ep.geojson")

#EQUIPAMIENTO DE ABASTO (MERCADOS PÚBLICOS)----
mercados<-st_read("https://datos.cdmx.gob.mx/dataset/6e008c6b-197c-49f2-899f-47b9a406596c/resource/39a74f11-55c5-4abf-aaf2-3558a9a281f7/download/mercados-pblicos.json")|>
  st_transform(crs = 32614)|>
  st_intersection(zona_interes)


#st_write(mercados,"mercados.geojson")

##EQUIPAMIENTO DE EDUCACIÓN
escuelas<-readxl:: read_excel("C:/Users/brenp/Downloads/concentrado_escuelas.xlsx") |>
  rename(LON = "Ubicación de la escuela-localidad al Oeste del Meridiano de Greenwich, expresada en grados", 
         LAT = "Ubicación de la escuela-localidad al norte del Ecuador, expresada en grados") |>
  mutate(coordenadas = paste0(LON, ",", LAT))|>
  mutate(LAT = as.numeric(LAT),LON = as.numeric(LON))|>
  filter(!is.na(LAT) & !is.na(LON))|>
  st_as_sf(coords = c("LON", "LAT"), crs = 4326)|>
  st_transform(32614)|>
  st_intersection(zona_interes)

#st_write(escuelas,"escuelas.geojson")


nivel<-escuelas|>
  group_by(Nivel.educativo)|>
  summarise(cuenta = n())|>
  st_drop_geometry()|>
  as.data.frame()


In [ ]:
##USO DE SUELO
predios <- st_read("C:/Users/brenp/Desktop/AIRBNB/csv_catastro/catastro2021_CIUDADDEMEXICO.shp")|>
  st_transform(32614)|>
  st_intersection(zona_interes)

hist(predios$vus, breaks = 10,                     
     main = "Distribución de VUS",
     xlab = "Valor de VUS",
     col = "gray")
abline(v = mean(predios$vus, na.rm = TRUE), col = "red", lwd = 2, lty = 2)
abline(v = median(predios$vus, na.rm = TRUE), col = "green", lwd = 2, lty = 2)
legend("topright", legend = c("Media", "Mediana"),col = c("red", "green"),lwd = 2)



#st_write(predios,"C:/Users/brenp/Desktop/PREDIO MH/predios.geojson")
